In [2]:
#final code
import pandas as pd
import numpy as np

# Set random seed for reproducibility
np.random.seed(42)

# Load the dataset
file_path = '/mnt/d/Algo Code/datasetNew.csv'
df = pd.read_csv(file_path)

# Extract resource constraints (last row)
constraint_row = df.iloc[-1, 3:].fillna(0).astype(int).values
df = df.iloc[:-1]

# Extract items and groups
items = []
groups = df['Group'].unique()
num_resources = len(constraint_row)

# Create a list of items with their resource usage and values
for _, row in df.iterrows():
    item = {
        'Group': row['Group'],
        'Item': row['Item'],
        'Value': row['Value'],
        'Resources': row[3:].values.astype(int)
    }
    items.append(item)

# Function to generate an initial population
def initialize_population(items, pop_size):
    population = []
    groups = set(item['Group'] for item in items)
    for _ in range(pop_size):
        individual = []
        for group in groups:
            group_items = [item for item in items if item['Group'] == group]
            individual.append(np.random.choice(group_items))
        population.append(individual)
    return population

# Function to evaluate the fitness of an individual
def evaluate_individual(individual, constraints):
    total_value = sum(item['Value'] for item in individual)
    total_resources = np.sum([item['Resources'] for item in individual], axis=0)
    valid = np.all(total_resources <= constraints)
    penalty = np.sum(np.maximum(total_resources - constraints, 0))  # Penalty for exceeding constraints
    return total_value - penalty if valid else -1

# Function to select the best individuals from the population
def select_population(population, fitness_scores, num_parents):
    parents_indices = np.argsort(fitness_scores)[-num_parents:]
    parents = [population[i] for i in parents_indices]
    return parents

# Function for crossover between two parents to create children
def crossover(parent1, parent2):
    crossover_point = np.random.randint(1, len(parent1))
    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]
    return child1, child2

# Function to mutate an individual
def mutate(individual, items):
    group = np.random.choice(list(set(item['Group'] for item in individual)))
    group_items = [item for item in items if item['Group'] == group]
    if len(group_items) > 1:
        idx = next(i for i, item in enumerate(individual) if item['Group'] == group)
        individual[idx] = np.random.choice(group_items)
    return individual

# Genetic Algorithm implementation with elitism
def genetic_algorithm(items, constraints, pop_size=100, num_generations=200, num_parents=20, mutation_rate=0.1, elitism_rate=0.1):
    population = initialize_population(items, pop_size)
    best_individual = None
    best_fitness = -1
    
    for generation in range(num_generations):
        fitness_scores = np.array([evaluate_individual(ind, constraints) for ind in population])
        
        # Print fitness values for this generation
        print(f"Generation {generation}:")
        for i, fitness in enumerate(fitness_scores):
            print(f"Individual {i}: Fitness Value = {fitness}")
        
        parents = select_population(population, fitness_scores, num_parents)
        
        # Generate new population through crossover and mutation
        new_population = []
        num_elites = int(pop_size * elitism_rate)
        if num_elites > 0:
            elite_indices = np.argsort(fitness_scores)[-num_elites:]
            new_population.extend([population[i] for i in elite_indices])
        
        while len(new_population) < pop_size:
            parent1, parent2 = np.random.choice(len(parents), 2, replace=False)
            child1, child2 = crossover(parents[parent1], parents[parent2])
            if np.random.rand() < mutation_rate:
                child1 = mutate(child1, items)
            if np.random.rand() < mutation_rate:
                child2 = mutate(child2, items)
            new_population.extend([child1, child2])
        
        population = new_population[:pop_size]
        
        # Update the best individual found so far
        fitness_scores = np.array([evaluate_individual(ind, constraints) for ind in population])
        best_idx = np.argmax(fitness_scores)
        if fitness_scores[best_idx] > best_fitness:
            best_fitness = fitness_scores[best_idx]
            best_individual = population[best_idx]
    
    return best_individual, best_fitness

# Function to generate an initial solution for Simulated Annealing
def initial_solution(items):
    groups = set(item['Group'] for item in items)
    solution = []
    for group in groups:
        group_items = [item for item in items if item['Group'] == group]
        solution.append(np.random.choice(group_items))
    return solution

# Function to generate a neighboring solution for Simulated Annealing
def neighbor_solution(solution, items):
    new_solution = solution.copy()
    groups = list(set(item['Group'] for item in solution))
    group = np.random.choice(groups)
    group_items = [item for item in items if item['Group'] == group]
    
    if len(group_items) > 1:
        new_item = np.random.choice(group_items)
        idx = next(i for i, item in enumerate(new_solution) if item['Group'] == group)
        new_solution[idx] = new_item

        # Additionally, swap items between different groups
        other_groups = [g for g in groups if g != group]
        if other_groups:
            other_group = np.random.choice(other_groups)
            other_group_items = [item for item in items if item['Group'] == other_group]
            if other_group_items:
                swap_idx = np.random.choice([i for i, item in enumerate(new_solution) if item['Group'] == other_group])
                new_solution[swap_idx] = np.random.choice(other_group_items)
    
    return new_solution

# Simulated Annealing algorithm implementation
def simulated_annealing(items, constraints, initial_temp=1000, cooling_rate=0.003, max_iterations=1000):
    current_solution = initial_solution(items)
    best_solution = current_solution
    current_value = evaluate_individual(current_solution, constraints)
    best_value = current_value
    temp = initial_temp
    
    for _ in range(max_iterations):
        new_solution = neighbor_solution(current_solution, items)
        new_value = evaluate_individual(new_solution, constraints)
        
        if new_value > current_value or np.random.rand() < np.exp((new_value - current_value) / temp):
            current_solution = new_solution
            current_value = new_value
            if new_value > best_value:
                best_solution = new_solution
                best_value = new_value
        
        temp *= (1 - cooling_rate)
    
    return best_solution, best_value

# Hybrid GA + SA approach
def hybrid_ga_sa(items, constraints, ga_params, sa_params):
    # Run Genetic Algorithm
    print("Running Genetic Algorithm...")
    ga_best_solution, ga_best_value = genetic_algorithm(items, constraints, **ga_params)
    
    # Run Simulated Annealing starting from the best GA solution
    print("Running Simulated Annealing...")
    sa_best_solution, sa_best_value = simulated_annealing(items, constraints, initial_temp=sa_params['initial_temp'], cooling_rate=sa_params['cooling_rate'], max_iterations=sa_params['max_iterations'])
    
    # Print the best values found in both algorithms
    print("\nGenetic Algorithm Best Value:", ga_best_value)
    print("Simulated Annealing Best Value:", sa_best_value)
    
    # Compare and return the best result from both approaches
    if sa_best_value > ga_best_value:
        return sa_best_solution, sa_best_value
    else:
        return ga_best_solution, ga_best_value

# Parameters
ga_params = {
    'pop_size': 100,
    'num_generations': 200,
    'num_parents': 20,
    'mutation_rate': 0.1,
    'elitism_rate': 0.1
}

sa_params = {
    'initial_temp': 1000,
    'cooling_rate': 0.003,
    'max_iterations': 1000
}

# Run Hybrid GA + SA
best_solution, best_value = hybrid_ga_sa(items, constraint_row, ga_params, sa_params)

# Output results
print("\nBest Solution:")
for item in best_solution:
    print(f"Group: {item['Group']}, Item: {item['Item']}, Value: {item['Value']}")

print("\nBest Total Value:", best_value)
print("Resource Constraints:", constraint_row)

/tmp/ipykernel_2319/2191586108.py:13: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  constraint_row = df.iloc[-1, 3:].fillna(0).astype(int).values


Running Genetic Algorithm...
Generation 0:
Individual 0: Fitness Value = 10221.0
Individual 1: Fitness Value = 10890.0
Individual 2: Fitness Value = 9997.0
Individual 3: Fitness Value = 9703.0
Individual 4: Fitness Value = 10067.0
Individual 5: Fitness Value = 11544.0
Individual 6: Fitness Value = 10189.0
Individual 7: Fitness Value = 10177.0
Individual 8: Fitness Value = 10476.0
Individual 9: Fitness Value = 10230.0
Individual 10: Fitness Value = 11237.0
Individual 11: Fitness Value = 10213.0
Individual 12: Fitness Value = 11057.0
Individual 13: Fitness Value = 9808.0
Individual 14: Fitness Value = 10329.0
Individual 15: Fitness Value = 10816.0
Individual 16: Fitness Value = 10651.0
Individual 17: Fitness Value = 10618.0
Individual 18: Fitness Value = 9897.0
Individual 19: Fitness Value = 10416.0
Individual 20: Fitness Value = 9553.0
Individual 21: Fitness Value = 10453.0
Individual 22: Fitness Value = 9822.0
Individual 23: Fitness Value = 11467.0
Individual 24: Fitness Value = 11193.